<a href="https://www.kaggle.com/code/thiagolimasantos/ps3-e17-eda-autogluon-models-lb-0-97882-top-6?scriptVersionId=134286969" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!pip install autogluon.tabular

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv
/kaggle/input/machine-failure-predictions/machine failure.csv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.2

In [2]:
############################### FEATURE ENGINER V2 #######################################################


# Read in the data
filepath = '/kaggle/input/playground-series-s3e17'

df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
original = pd.read_csv('/kaggle/input/machine-failure-predictions/machine failure.csv', index_col=[0])
target_col = 'Machine failure'
# Stack training and original data
train = pd.concat([df_train, original])
test = df_test.copy()

X_train = train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = train[f'{target_col}'].reset_index(drop=True)
X_test = test.reset_index(drop=True)


def create_features(df):
    df["Temperature ratio"] = df['Process temperature [K]'] / df['Air temperature [K]']
    df['Power']= df['Torque [Nm]']*df['Rotational speed [rpm]']
    df['Torque * Tool wear'] = df['Torque [Nm]'] * df['Tool wear [min]']
    df['TotalFailures'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)
    new_cols = [
        #'Temperature difference [K]', 
        'Power', 
        'Temperature ratio',
        'Torque * Tool wear','TotalFailures']
    return df, new_cols

def rename(df):
    df.columns = df.columns.str.replace('\[.*?\]', '', regex=True)
    df.columns = df.columns.str.strip()
    # df.columns = df.columns.str.replace(' ', '')
    
    return df

X_train, _ = create_features(X_train)
X_test, new_cols = create_features(X_test)
# Rename
X_train = rename(X_train)
X_test = rename(X_test)

In [3]:
from autogluon.tabular import TabularPredictor, TabularDataset
train = X_train
train["label"] = y_train
train = TabularDataset(train)

In [4]:
excluded_model_types = ['KNN', 'NN_TORCH']
predictor2 = TabularPredictor(label="label", problem_type='binary',verbosity=-1,
                             eval_metric='log_loss',sample_weight='balance_weight').fit(train,presets="best_quality",excluded_model_types=excluded_model_types,
                                                                    auto_stack=True,save_space=True,
                                                                    num_stack_levels=2, num_bag_folds=3,num_bag_sets=6,time_limit=1800)
# ['accuracy', 'acc', 'balanced_accuracy', 'mcc', 'roc_auc_ovo_macro', 'log_loss', 'nll', 'pac', 'pac_score', 'quadratic_kappa', 'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro',
#  'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted']

In [5]:
predictor2.leaderboard()
#  -0.016092 

                      model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L4  -0.015105      70.052020  1628.970969                0.003723           2.922293            4       True         31
1       WeightedEnsemble_L3  -0.016383      47.964760  1277.493315                0.003462           3.437823            3       True         22
2    NeuralNetFastAI_BAG_L2  -0.018488      27.995533   918.186843                2.537787         151.658978            2       True         19
3       WeightedEnsemble_L2  -0.018710      22.114717   571.143547                0.003517           3.398674            2       True         11
4     ExtraTreesGini_BAG_L3  -0.020099      55.788869  1431.462828                5.126537          26.090716            3       True         28
5   RandomForestEntr_BAG_L3  -0.020875      55.003169  1481.945106                4.340837          76.572994            3       T

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L4,-0.015105,70.052020,1628.970969,0.003723,2.922293,4,True,31
1,WeightedEnsemble_L3,-0.016383,47.964760,1277.493315,0.003462,3.437823,3,True,22
2,NeuralNetFastAI_BAG_L2,-0.018488,27.995533,918.186843,2.537787,151.658978,2,True,19
3,WeightedEnsemble_L2,-0.018710,22.114717,571.143547,0.003517,3.398674,2,True,11
4,ExtraTreesGini_BAG_L3,-0.020099,55.788869,1431.462828,5.126537,26.090716,3,True,28
5,RandomForestEntr_BAG_L3,-0.020875,55.003169,1481.945106,4.340837,76.572994,3,True,26
6,ExtraTreesEntr_BAG_L3,-0.021104,55.621326,1429.761200,4.958994,24.389088,3,True,29
7,NeuralNetFastAI_BAG_L1,-0.021415,2.425448,278.686800,2.425448,278.686800,1,True,8
8,RandomForestGini_BAG_L3,-0.021876,55.039463,1488.393146,4.377131,83.021034,3,True,25
9,RandomForestEntr_BAG_L2,-0.026020,29.989874,855.050297,4.532128,88.522433,2,True,15


In [6]:
#y_pred_auc = predictor.predict_proba(X_test, as_pandas=False, as_multiclass=False)
y_pred_logloss = predictor2.predict_proba(X_test, as_pandas=False, as_multiclass=False)

In [7]:
y_pred_logloss

array([0.0001955 , 0.00021024, 0.00018685, ..., 0.00019917, 0.00091653,
       0.00018626])

In [8]:

#models = ['WeightedEnsemble_L3','WeightedEnsemble_L2']

# Inicialize uma matriz vazia para armazenar as previsões de cada modelo
#predictions = []

# Faça previsões para cada modelo
#for models in models:
    # Realize a previsão para o modelo atual
#    model_pred = predictor.predict_proba(X_test, model=models,as_pandas=False, as_multiclass=False)
    # Adicione as previsões do modelo atual à matriz
#    predictions.append(model_pred)

# Calcule a média das previsões de todos os modelos
#average_prediction = np.mean(predictions, axis=0)

# Exiba a média das previsões
#print("Average Prediction:", average_prediction)
#Average Prediction: [0.03733805 0.06888288 0.0210157  ... 0.04945559 0.02032589 0.03096515]

In [9]:


#ypredmedia = (y_pred_auc+y_pred_logloss+average_prediction)/3
def make_submission(test_predss, prefix=''):
    sub = pd.read_csv(os.path.join(filepath, 'sample_submission.csv'))
    sub[f'{target_col}'] = test_predss
    sub.to_csv(f'{prefix}submission.csv', index=False)
    return  sub
#sub = make_submission(np.where(ypredmedia >= 0.50, 1, 0), prefix='')
sub = make_submission(y_pred_logloss, prefix='')
sub

,id,Machine failure
0,136429,0.000196
1,136430,0.000210
2,136431,0.000187
3,136432,0.000415
4,136433,0.000190
...,...,...
90949,227378,0.000239
90950,227379,0.005999
90951,227380,0.000199
90952,227381,0.000917
